## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '../..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

In [6]:
import glob
import csv
from tqdm.auto import tqdm

from entity_embed.data_utils.utils import Enumerator

row_dict = {}
id_enumerator = Enumerator()
left_id_set = set()
right_id_set = set()
rows_total = 1363 + 3226
clusters_total = 1300

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/amazon_google_exp_data/tableA.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'left-{int(row["id"])}']
            row['source'] = 'left'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/amazon_google_exp_data/tableB.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'right-{int(row["id"])}']
            row['source'] = 'right'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [7]:
train_true_pair_set = set()
valid_true_pair_set = set()
test_true_pair_set = set()

with open(f'{home_dir}/Downloads/amazon_google_exp_data/train.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            train_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/amazon_google_exp_data/valid.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            valid_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/amazon_google_exp_data/test.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            test_true_pair_set.add((id_left, id_right))
        
display(('train_true_pair_set', len(train_true_pair_set)))
display(('valid_true_pair_set', len(valid_true_pair_set)))
display(('test_true_pair_set', len(test_true_pair_set)))

('train_true_pair_set', 699)

('valid_true_pair_set', 234)

('test_true_pair_set', 234)

## Preprocess

In [8]:
attr_list = ['title', 'manufacturer', 'price']

In [9]:
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    }
}

In [13]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

17:11:29 INFO:For attr=title, computing actual max_str_len
17:11:29 INFO:For attr=title, using actual_max_str_len=26
17:11:29 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:11:33 INFO:For attr=manufacturer, computing actual max_str_len
17:11:33 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:11:33 INFO:For attr=manufacturer, using actual_max_str_len=16
17:11:33 INFO:For attr=price, computing actual max_str_len
17:11:33 INFO:actual_max_str_len=11 must be pair to enable NN pooling. Updating to 12
17:11:33 INFO:For attr=price, using actual_max_str_len=12


{'title': NumericalizeInfo(attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=26, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': NumericalizeInfo(attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"',

In [14]:
from entity_embed import PairwiseDataModule

datamodule = PairwiseDataModule(
    row_dict=row_dict,
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    train_true_pair_set=train_true_pair_set,
    valid_true_pair_set=valid_true_pair_set,
    test_true_pair_set=test_true_pair_set,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = 'f1-amzn-googl'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

17:11:33 INFO:GPU available: True, used: True
17:11:33 INFO:TPU available: None, using: 0 TPU cores
17:11:33 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

17:11:35 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.4 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
5.7 M     Trainable params
1.7 M     Non-trainable params
7.4 M     Total params


17:11:36 WARNING:true_pair_set has 82 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [18]:
model.blocker_net.get_signature_weights()

{'title': 0.40408724546432495,
 'semantic_title': 0.35267871618270874,
 'manufacturer': 0.1278088092803955,
 'price': 0.11543220281600952}

In [19]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.420863309352518,
 'valid_f1_at_0.5': 0.7553366174055828,
 'valid_f1_at_0.7': 0.8340080971659919,
 'valid_f1_at_0.9': 0.7193877551020408,
 'valid_pair_entity_ratio_at_0.3': 1.9212253829321664,
 'valid_pair_entity_ratio_at_0.5': 0.8205689277899344,
 'valid_pair_entity_ratio_at_0.7': 0.5689277899343544,
 'valid_pair_entity_ratio_at_0.9': 0.34573304157549234,
 'valid_precision_at_0.3': 0.26651480637813213,
 'valid_precision_at_0.5': 0.6133333333333333,
 'valid_precision_at_0.7': 0.7923076923076923,
 'valid_precision_at_0.9': 0.8924050632911392,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9829059829059829,
 'valid_recall_at_0.7': 0.8803418803418803,
 'valid_recall_at_0.9': 0.6025641025641025}

## Testing

In [20]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.4152617568766637,
  'test_f1_at_0.5': 0.7240829346092504,
  'test_f1_at_0.7': 0.832,
  'test_f1_at_0.9': 0.7684729064039408,
  'test_pair_entity_ratio_at_0.3': 1.9370932754880694,
  'test_pair_entity_ratio_at_0.5': 0.8524945770065075,
  'test_pair_entity_ratio_at_0.7': 0.5770065075921909,
  'test_pair_entity_ratio_at_0.9': 0.37310195227765725,
  'test_precision_at_0.3': 0.2620380739081747,
  'test_precision_at_0.5': 0.5776081424936387,
  'test_precision_at_0.7': 0.7819548872180451,
  'test_precision_at_0.9': 0.9069767441860465,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 0.9700854700854701,
  'test_recall_at_0.7': 0.8888888888888888,
  'test_recall_at_0.9': 0.6666666666666666}]

## Testing manually 

In [21]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [22]:
embedding_size = model.blocker_net.embedding_size

In [23]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [24]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 291 ms, sys: 0 ns, total: 291 ms
Wall time: 44.3 ms


In [25]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 265 ms, sys: 0 ns, total: 265 ms
Wall time: 29.6 ms


In [26]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

0.5770065075921909

In [27]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, test_true_pair_set)
precision, recall

(0.7819548872180451, 0.8888888888888888)

In [28]:
f1_score(precision, recall)

0.832

In [29]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

58

In [30]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

26

In [31]:
cos_similarity = lambda a, b: np.dot(a, b)

In [32]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.4858476,
 {'id': 866,
  'title': 'system care professional',
  'manufacturer': 'avanquest',
  'price': '49 . 95',
  'source': 'left'},
 {'id': 1566,
  'title': 'avanquest usa llc system care professional',
  'manufacturer': '',
  'price': '43 . 32',
  'source': 'right'})

(0.69045776,
 {'id': 395,
  'title': 'world history ( win / mac ) ( jewel case )',
  'manufacturer': 'fogware publishing',
  'price': '9 . 99',
  'source': 'left'},
 {'id': 4269,
  'title': 'high school world history ( pc / mac ) fogware',
  'manufacturer': '',
  'price': '9 . 99',
  'source': 'right'})

(0.6412056,
 {'id': 1206,
  'title': 'contentbarrier x4 10 . 4 single user ( mac )',
  'manufacturer': 'intego',
  'price': '49 . 99',
  'source': 'left'},
 {'id': 1900,
  'title': 'intego contentbarrier x4 10 . 4',
  'manufacturer': '',
  'price': '54 . 99',
  'source': 'right'})

(0.51732856,
 {'id': 345,
  'title': 'portfolio media kit be syst recovery 7 . 0 win small business ed',
  'manufacturer': 'symantec',
  'price': '50 . 0',
  'source': 'left'},
 {'id': 2883,
  'title': 'symantec 11859201 be sys recovery 7 . 0 win sbs ed media cd m / l',
  'manufacturer': '',
  'price': '31 . 98',
  'source': 'right'})

(0.5302396,
 {'id': 1360,
  'title': 'dragon naturally speaking standard v9',
  'manufacturer': 'nuance - communications - inc .',
  'price': '99 . 99',
  'source': 'left'},
 {'id': 1973,
  'title': 'nuance communications inc . dragon ns standard v9',
  'manufacturer': 'nuance - communications - inc .',
  'price': '92 . 51',
  'source': 'right'})

(0.64657986,
 {'id': 404,
  'title': 'cosmi rom07524 print perfect business cards dvd',
  'manufacturer': 'cosmi',
  'price': '',
  'source': 'left'},
 {'id': 4253,
  'title': 'print perfect business cards dvd ( pc ) cosmi',
  'manufacturer': '',
  'price': '29 . 99',
  'source': 'right'})

(0.63293105,
 {'id': 726,
  'title': 'simply put software data eliminator',
  'manufacturer': 'simply - put - software',
  'price': '',
  'source': 'left'},
 {'id': 4151,
  'title': 'simply put software llc de905 - s data elminator ( win 95 98 me nt 2000 xp )',
  'manufacturer': 'simply - put - software',
  'price': '34 . 97',
  'source': 'right'})

(0.62741977,
 {'id': 812,
  'title': 'extensis smartscale 1 - user pxe - 11433 )',
  'manufacturer': 'extensis corporation',
  'price': '',
  'source': 'left'},
 {'id': 4160,
  'title': 'onone software pxe - 11433 pxl smartscale elect 1 - user english mac / win',
  'manufacturer': 'onone software',
  'price': '158 . 99',
  'source': 'right'})

(0.6501619,
 {'id': 168,
  'title': 'autodesk discreet combustion 4 windows )',
  'manufacturer': 'autodesk',
  'price': '',
  'source': 'left'},
 {'id': 3527,
  'title': 'autodesk combustion 4 . 0 compositing software win compositing software',
  'manufacturer': '',
  'price': '889 . 0',
  'source': 'right'})

(0.6109418,
 {'id': 97,
  'title': 'tournament poker 2005',
  'manufacturer': 'eagle games',
  'price': '20 . 99',
  'source': 'left'},
 {'id': 3000,
  'title': "eagle games egl 150 tournament poker no limit texas hold ' em",
  'manufacturer': 'eagle games',
  'price': '7 . 95',
  'source': 'right'})